In [2]:
import pickle
import pandas as pd
import numpy as np

import xgboost as xgb
import lightgbm as lgb

from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing

from keras.models import load_model
from keras.utils import np_utils

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto(
    gpu_options=tf.GPUOptions(
#         visible_device_list="1", 
        allow_growth=True
    )
)

set_session(tf.Session(config=config))

In [3]:
# read data
with open('../data/HCD35_pos.pickle', mode='rb') as fp:
    df_3 = pickle.load(fp)
    
df = pd.concat([
    df_3,
    pd.read_csv('../data/decomp_pos.csv').drop('Subclass', axis=1)
], axis=1)

In [4]:
objective = df.Subclass
le = preprocessing.LabelEncoder()
objective = le.fit_transform(objective)

features = df.drop('Subclass', axis=1)

In [5]:
random_state=np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    objective,
    test_size=0.2
)

In [6]:
# transform  for keras's target label
y_train_for_keras = np_utils.to_categorical(y_train)
y_test_for_keras = np_utils.to_categorical(y_test)

In [7]:
d = {}

In [9]:
xgb_model = pickle.load(open('../model/xgb_pos_HCD35+decomp.sav', 'rb'))
xgb_result = xgb_model.predict_proba(X_test)
print(xgb_model.score(X_test, y_test))
d['Xgboost'] = [round(xgb_model.score(X_test, y_test)*100, 2)]

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.8065476190476191


/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [10]:
rf_model = pickle.load(open('../model/rf_pos_HCD35+decomp.sav', 'rb'))
rf_result = rf_model.predict_proba(X_test)
print(rf_model.score(X_test, y_test))
d['Random Forest'] = [round(rf_model.score(X_test, y_test)*100, 2)]

0.6994047619047619


In [12]:
lgb_model = pickle.load(open('../model/lgbm_pos_HCD35+decomp.sav', 'rb'))
lgb_result = lgb_model.predict_proba(X_test)
print(lgb_model.score(X_test, y_test))
d['LightGBM'] = [round(lgb_model.score(X_test, y_test)*100, 2)]

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.8363095238095238


/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [13]:
keras_model  = load_model('../model/Keras_pos_HCD35+decomp.h5')
keras_result = keras_model.predict(X_test)
d['Keras'] = [round(keras_model.evaluate(X_test, y_test_for_keras, verbose=0)[1]*100, 2)]

In [14]:
o = {}

In [15]:
o['XGBoost + Ramdom Forest'] = [round((sum(np.argmax((xgb_result+rf_result) , axis=1) == y_test)/len(y_test))*100, 2)]
o['XGBoost + LightGBM'] = [round((sum(np.argmax((xgb_result+lgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]
o['XGBoost + Keras'] = [round((sum(np.argmax((xgb_result+keras_result) , axis=1) == y_test)/len(y_test))*100, 2)]
o['Random Forest + LightGBM'] = [round((sum(np.argmax((rf_result+lgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]
o['Random Forest + Keras'] = [round((sum(np.argmax((rf_result+keras_result) , axis=1) == y_test)/len(y_test))*100, 2)]
o['LightGBM+Keras'] = [round((sum(np.argmax((keras_result+lgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]

o['XGBoost + Random Forest + LightGBM'] = [round((sum(np.argmax((rf_result+lgb_result+xgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]
o['XGBoost + Random Forest + Keras'] = [round((sum(np.argmax((rf_result+keras_result+xgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]
o['XGBoost + LightGBM + Keras'] = [round((sum(np.argmax((keras_result+lgb_result+xgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]
o['Random Forest + LightGBM + Keras'] = [round((sum(np.argmax((rf_result+keras_result+lgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]

o['XGBoost + Random Forest + LightGBM + Keras'] = [round((sum(np.argmax((rf_result+keras_result+lgb_result+xgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]

In [16]:
pd.DataFrame(o).T
# .to_csv('../result/HCD35_pos.csv')

,0
XGBoost + Ramdom Forest,80.65
XGBoost + LightGBM,83.04
XGBoost + Keras,76.79
Random Forest + LightGBM,82.74
Random Forest + Keras,72.02
LightGBM+Keras,82.44
XGBoost + Random Forest + LightGBM,82.74
XGBoost + Random Forest + Keras,78.57
XGBoost + LightGBM + Keras,84.52
Random Forest + LightGBM + Keras,82.44


In [17]:
# read data
with open('../data/HCD35_neg.pickle', mode='rb') as fp:
    df_3 = pickle.load(fp)
    
df = pd.concat([
    df_3,
    pd.read_csv('../data/decomp_neg.csv').drop('Subclass', axis=1)
], axis=1)

In [18]:
objective = df.Subclass
le = preprocessing.LabelEncoder()
objective = le.fit_transform(objective)

features = df.drop('Subclass', axis=1)

In [19]:
random_state=np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    objective,
    test_size=0.2
)

In [20]:
# transform  for keras's target label
y_train_for_keras = np_utils.to_categorical(y_train)
y_test_for_keras = np_utils.to_categorical(y_test)

In [21]:
d = {}

In [22]:
xgb_model = pickle.load(open('../model/xgb_ng_HCD35+decomp.sav', 'rb'))
xgb_result = xgb_model.predict_proba(X_test)
print(xgb_model.score(X_test, y_test))
d['Xgboost'] = [round(xgb_model.score(X_test, y_test)*100, 2)]

0.7872340425531915


/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [23]:
rf_model = pickle.load(open('../model/rf_ng_HCD35+decomp.sav', 'rb'))
rf_result = rf_model.predict_proba(X_test)
print(rf_model.score(X_test, y_test))
d['Random Forest'] = [round(rf_model.score(X_test, y_test)*100, 2)]

0.6914893617021277


In [24]:
lgb_model = pickle.load(open('../model/lgbm_ng_HCD35+decomp.sav', 'rb'))
lgb_result = lgb_model.predict_proba(X_test)
print(lgb_model.score(X_test, y_test))
d['LightGBM'] = [round(lgb_model.score(X_test, y_test)*100, 2)]

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.723404255319149


/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [25]:
keras_model  = load_model('../model/Keras_ng_HCD35+decomp.h5')
keras_result = keras_model.predict(X_test)
d['Keras'] = [round(keras_model.evaluate(X_test, y_test_for_keras, verbose=0)[1]*100, 2)]

In [26]:
o = {}

In [27]:
o['XGBoost + Ramdom Forest'] = [round((sum(np.argmax((xgb_result+rf_result) , axis=1) == y_test)/len(y_test))*100, 2)]
o['XGBoost + LightGBM'] = [round((sum(np.argmax((xgb_result+lgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]
o['XGBoost + Keras'] = [round((sum(np.argmax((xgb_result+keras_result) , axis=1) == y_test)/len(y_test))*100, 2)]
o['Random Forest + LightGBM'] = [round((sum(np.argmax((rf_result+lgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]
o['Random Forest + Keras'] = [round((sum(np.argmax((rf_result+keras_result) , axis=1) == y_test)/len(y_test))*100, 2)]
o['LightGBM+Keras'] = [round((sum(np.argmax((keras_result+lgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]

o['XGBoost + Random Forest + LightGBM'] = [round((sum(np.argmax((rf_result+lgb_result+xgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]
o['XGBoost + Random Forest + Keras'] = [round((sum(np.argmax((rf_result+keras_result+xgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]
o['XGBoost + LightGBM + Keras'] = [round((sum(np.argmax((keras_result+lgb_result+xgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]
o['Random Forest + LightGBM + Keras'] = [round((sum(np.argmax((rf_result+keras_result+lgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]

o['XGBoost + Random Forest + LightGBM + Keras'] = [round((sum(np.argmax((rf_result+keras_result+lgb_result+xgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]

In [28]:
pd.DataFrame(o).T
# to_csv('../result/HCD35_neg.csv')

,0
XGBoost + Ramdom Forest,79.79
XGBoost + LightGBM,75.53
XGBoost + Keras,72.34
Random Forest + LightGBM,74.47
Random Forest + Keras,67.02
LightGBM+Keras,73.40
XGBoost + Random Forest + LightGBM,77.66
XGBoost + Random Forest + Keras,74.47
XGBoost + LightGBM + Keras,73.40
Random Forest + LightGBM + Keras,77.66
